In [1]:
import os.path as osp

In [2]:
def make_datapath_list(rootpath):
    imgpath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
    annopath_template = osp.join(rootpath, "Annotations", "%s.xml")

    train_id_names = osp.join(rootpath + "ImageSets/Main/train.txt")
    val_id_names = osp.join(rootpath + "ImageSets/Main/val.txt")
    
    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)

    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.appen(img_path)
        val_anno_list.appen(anno_path)

    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [ ]:
rootpath = "./data/"

train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

print(train_img_list[0])